In [214]:
import pandas as pd
import requests

In [215]:
df = pd.read_csv("../../camada_bronze/acao_petroleo.csv", delimiter=";")

In [216]:
display(df)

,Date,Open,High,Low,Close,Volume
0,2010-01-04 16:56:00,"37,56762146","37,93762146","37,43762146","37,93762146","13303743,87"
1,2010-01-05 16:56:00,"38,15421972","38,20421972","37,57421972","37,77421972","21396519,48"
2,2010-01-06 16:56:00,"38,10535714","38,80535714","38,10535714","38,80535714","18720547,15"
3,2010-01-07 16:56:00,"36,54592902","36,72592902","36,34592902","36,42592902","10964646,19"
4,2010-01-08 16:56:00,"35,24778748","35,47778748","34,94778748","35,03778748","14624078,31"
...,...,...,...,...,...,...
3647,2024-12-20 16:56:00,"39,15559512","39,34559512","38,70559512","38,84559512","59277431,74"
3648,2024-12-23 16:56:00,"39,46015699","39,51015699","39,18015699","39,32015699","43785532,81"
3649,2024-12-26 16:56:00,"37,72760865","38,09760865","37,69760865","37,86760865","22920777,53"
3650,2024-12-27 16:56:00,"36,4068181","36,4068181","36,0168181","36,0668181","24167282,36"


In [217]:
df.isna().sum()

Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [218]:
df = df.fillna(0)

In [219]:
df = df.drop_duplicates()

In [220]:
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

In [221]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3652 entries, 0 to 3651
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    3652 non-null   object
 1   Open    3652 non-null   object
 2   High    3652 non-null   object
 3   Low     3652 non-null   object
 4   Close   3652 non-null   object
 5   Volume  3652 non-null   object
dtypes: object(6)
memory usage: 171.3+ KB


In [222]:
colunas = ["Open","High","Low","Close","Volume"]

df[colunas] = df[colunas].map(lambda x: x.replace(",","."))

In [223]:
df[colunas] = df[colunas].apply(lambda x: pd.to_numeric(x))

In [224]:
df["Date"] = pd.to_datetime(df["Date"])

In [225]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3652 entries, 0 to 3651
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    3652 non-null   datetime64[ns]
 1   Open    3652 non-null   float64       
 2   High    3652 non-null   float64       
 3   Low     3652 non-null   float64       
 4   Close   3652 non-null   float64       
 5   Volume  3652 non-null   float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 171.3 KB


In [226]:
df = df.rename(columns={"Date":"Datetime"})

In [227]:
df["Date"] = df["Datetime"].dt.date

In [228]:
last_column = df.columns[-1]
df = df[[last_column] + [col for col in df.columns if col != last_column]]

In [229]:
#Função para coletar dados da cotação do dólar em determinado tempo através de uma API

def get_cotacao_dolar(data):
    
    if isinstance(data, str):
        data = pd.to_datetime(data)

    #Converte o Timestamp para string no formato DD-MM-YYYY
    data_inicial = f"{data.iloc[0].day}/{data.iloc[0].month}/{data.iloc[0].year}"
    data_final = f"{data.iloc[-1].day}/{data.iloc[-1].month}/{data.iloc[-1].year}"

    url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.1/dados?dataInicial={data_inicial}&dataFinal={data_final}&formato=json"
    response = requests.get(url)
    
    if response.status_code == 200:
        try:
            data_dolar = response.json()
            # Verificar se a resposta está vazia
            if data_dolar:
                return data_dolar
            else:
                print(f"Nenhum dado encontrado entre as datas {data_inicial} e {data_final}")
                return None  # Caso não tenha dados para aquela data
        except Exception as e:
            print(f"Erro ao tentar decodificar JSON entre as datas {data_inicial} e {data_final}: {e}")
    else:
        print(f"Erro na requisição entre as datas {data_inicial} e {data_final}, código de status: {response.status_code}")
    return None


In [230]:
df_dolar = pd.DataFrame(get_cotacao_dolar(df["Date"]))

In [231]:
df_dolar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3766 entries, 0 to 3765
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   data    3766 non-null   object
 1   valor   3766 non-null   object
dtypes: object(2)
memory usage: 59.0+ KB


In [232]:
df_dolar["valor"] = df_dolar["valor"].astype(float)

In [233]:
df["Valor_Dolar"] = df["Close"]/df_dolar["valor"]

In [234]:
# Adicionar uma nova coluna 'PriceR$' para o preço em reais, ignorando datas com erro
#df['PriceR$'] = df['Date'].apply(lambda x: get_cotacao_dolar(x)).multiply(df['Price'])

In [235]:
display(df)

,Date,Datetime,Open,High,Low,Close,Volume,Valor_Dolar
0,2010-01-04,2010-01-04 16:56:00,37.567621,37.937621,37.437621,37.937621,13303743.87,22.005581
1,2010-01-05,2010-01-05 16:56:00,38.154220,38.204220,37.574220,37.774220,21396519.48,21.927335
2,2010-01-06,2010-01-06 16:56:00,38.105357,38.805357,38.105357,38.805357,18720547.15,22.382971
3,2010-01-07,2010-01-07 16:56:00,36.545929,36.725929,36.345929,36.425929,10964646.19,20.918813
4,2010-01-08,2010-01-08 16:56:00,35.247787,35.477787,34.947787,35.037787,14624078.31,20.148239
...,...,...,...,...,...,...,...,...
3647,2024-12-20,2024-12-20 16:56:00,39.155595,39.345595,38.705595,38.845595,59277431.74,7.123841
3648,2024-12-23,2024-12-23 16:56:00,39.460157,39.510157,39.180157,39.320157,43785532.81,7.206377
3649,2024-12-26,2024-12-26 16:56:00,37.727609,38.097609,37.697609,37.867609,22920777.53,6.977118
3650,2024-12-27,2024-12-27 16:56:00,36.406818,36.406818,36.016818,36.066818,24167282.36,6.597186


In [ ]:
df.to_csv('../../camada_prata/eyac2_clean.csv', index=False)